In [1]:
import pandas as pd
import numpy as np
import os
from math import *

In [2]:
# Read the size of the file
def get_dir_size(dir): 
    size = 0 
    for root, dirs, files in os.walk(dir): 
        size += sum([os.path.getsize(root + '/'+ name) for name in files]) 
    return size

In [3]:
# Determine whether the file is larger than 200M as the standard for whether convergence is complete
def get_origin_data (i,dirpath):
    size = get_dir_size(dirpath) 
    print (i,size) 
    if size >= 209715200: 
        a = 1
    else:
        a = 0
    return a 

In [4]:
def read_file (file):
    data = pd.read_csv(file)
    return data

In [5]:
# Gets the lattice constant of the CONTCAR file
def get_lattice(data,df,i):
    b = [] 
    for i1 in [1,2,3]: 
        a = data.iloc[i1,0] 
        a = a.split(' ') 
        b = b+a 
    while '' in b: 
        b.remove('')
    df['{}'.format(i)] = b 
    return df,b 

In [6]:
# Gets the average lattice constant
def get_mean_lattice(df): 
    df = df.astype('float') 
    df['均值']=df.mean(axis=1) 
    mean = df.iloc[:,-1] 
    print ('均值为：{}'.format(mean)) 
    return mean 

In [7]:
# Gets the average lattice constant
def get_mean_abc(mean):
    a = str(mean.iloc[0])+' '+str(mean.iloc[1])+' '+str(mean.iloc[2])
    b = str(mean.iloc[3])+' '+str(mean.iloc[4])+' '+str(mean.iloc[5])
    c = str(mean.iloc[6])+' '+str(mean.iloc[7])+' '+str(mean.iloc[8])
    return a,b,c 

In [8]:
# Replacement lattice constant
def get_train_contcar(a,b,c,data):
    data.loc[1,:]=a
    data.loc[2,:]=b
    data.loc[3,:]=c
    return data 

In [9]:
# Write a new CONTCAR
def write_contcar(i,data,path):
    data.to_csv(path + '/CONTCAR{}'.format(i),index=False)

In [10]:
# create new dir
def makedir(path1,path2,path3,path4):
        if os.path.exists(path1) == 0: 
            os.mkdir(path1)
        if os.path.exists(path2) == 0:
            os.mkdir(path2)
        if os.path.exists(path3) == 0:
            os.mkdir(path3)
        if os.path.exists(path4) == 0:
            os.mkdir(path4)

In [11]:
# convert cif and get lattice constant
def get_abc(file,path4,i2):
    os.system('ase convert {} {}/{}.cif'.format(file,path4,i2))
    data = pd.read_csv('{}/{}.cif'.format(path4,i2),error_bad_lines = False)
    ls = []
    for i in range(2,5):
        l = data.iloc[i,0] 
        l = l.split(' ') 
        ls = ls+l 
    while '' in ls: 
        ls.remove('')
    a = ls[1]
    b = ls[3]
    c = ls[5]
    return a,b,c

In [12]:
# calculate the surface strain
def get_strain(a,b,c):
    l = sqrt((pow(a,2)+pow(b,2)))
    m = sqrt((pow(a,2)+pow(c,2)))
    n = sqrt((pow(b,2)+pow(c,2)))
    p = (l + m + n)/2
    s = sqrt(p * (p-l) * (p-m) * (p-n))
    return s

In [13]:
# calculate the surface strain via ptcu3
def get_strain_via_ptcu3(a,b,c):
    s_ptcu3 = get_strain(7.45216 ,7.45215 , 7.45216)
    suface_strain = (get_strain(a,b,c) - s_ptcu3) / s_ptcu3
    return suface_strain

In [15]:
# get the oszicar
def get_oszicar(file_OSZICAR):
    with open(file_OSZICAR, 'r', encoding='utf-8') as f: 
        lines = f.readlines()  
        last_line = lines[-1]  
    return last_line 

In [16]:
# get the atom of contcar
def get_train_atom(data):
    atom = data.iloc[4,0]
    counts = data.iloc[5,0]
    atom = atom.split(' ')
    counts = counts.split(' ')
    while '' in atom: 
        atom.remove('')
    while '' in counts: 
        counts.remove('')
    return atom,counts

In [17]:
# get e0
def get_e0(last_line , atom , counts , data_energy):
    last_line = last_line.split(' ') 
    for i in range (len(last_line)): 
        if last_line[i] == 'E0=':
            E0 = float(last_line[i+1])
    for i in range(len(atom)):
        E0 = E0 - data_energy.loc[atom[i],0] * int(counts[i])
    return E0 

In [18]:
# write num and e0
def write_csv_e0(i,E0,path2):
    csv_data = [str(i),str(E0)] 
    with open(path2+'/id_prop.csv', 'a', encoding='utf-8') as f:
        f.write(','.join(csv_data)+'\n') 

In [19]:
# write num and surface strain
def write_csv_strain(i,y,path2):
    csv_data = [str(i),str(y)] 
    with open(path3+'/id_prop.csv', 'a', encoding='utf-8') as f:
        f.write(','.join(csv_data)+'\n') 

In [20]:
def main(n,path1,path2,path3,path4,data_energy):
    df = pd.DataFrame() 
    x = []
    y = [] 
    makedir(path1,path2,path3,path4) 
    for i in range(n): 
        dirpath = f'results{i}'
        a = get_origin_data(i,dirpath) 
        if a == 1: 
            x.append(i)
            file = dirpath + '/CONTCAR' 
            data = read_file(file) 
            df,dt = get_lattice(data,df,i) 
            a1,b1,c1 = get_abc(file,path4,i)
            suface_strain = get_strain_via_ptcu3(float(a1),float(b1),float(c1))
            y.append(suface_strain)
    mean = get_mean_lattice(df)
    a,b,c = get_mean_abc(mean) 
    print (x) 
    coordinate = pd.read_csv('POSCAR').iloc[7:39,:] 
    for i in x: 
        file = f'results{i}'
        print (file) 
        data = read_file(f'{file}/CONTCAR') 
        data = get_train_contcar(a,b,c,data)
        data.iloc[7:39,:] = coordinate 
        atom,counts = get_train_atom(data)
        last_line = get_oszicar(f'{file}/OSZICAR')
        E0 = get_e0(last_line,atom,counts,data_energy)
        write_contcar(i,data,path1) 
        os.system('ase convert {}/CONTCAR{} {}/{}.cif'.format(path1,i,path2,i))
        write_csv_e0(i,str(E0),path2) 
        write_csv_strain(i,y.pop(0),path3)

In [ ]:
n = 651
path1 = r'results_after' 
path2 = r'results_cif' 
path3 = r'results_suface'
path4 = r'results_initial_cif'
data_energy = pd.DataFrame([-6.0982,-9.1198,-8.9174,-7.03679,-5.4686,-3.72838,
                               -2.2218,-8.2377852,-2.906231,-2.55672,-10.932452],
                              index = ['Pt','Cr','Mn','Co','Ni','Cu','Zn','Fe','Ga','In','Mo'])
main(n,path1,path2,path3,path4,data_energy) 